# 지도학습 분류 미리보기

## 학습 목표
- 이진분류와 다중분류의 개념 이해
- K-최근접 이웃(KNN) 알고리즘의 작동 원리 파악
- 데이터 전처리와 정규화의 중요성 학습
- 머신러닝 모델의 기본 파이프라인 이해

In [ ]:
# 필요한 라이브러리 import
import numpy as np        
import pandas as pd        
import matplotlib.pyplot as plt  

In [ ]:
# 한글 폰트 설정 (matplotlib에서 한글이 깨지지 않도록)
import matplotlib.font_manager as fm  # 폰트 관리 모듈
import matplotlib

# Windows 시스템의 한글 폰트 경로 설정
font_path = 'C:\\Windows\\Fonts\\H2HDRM.TTF'  # 한글 폰트 파일 경로
font = fm.FontProperties(fname=font_path).get_name()  # 폰트 이름 추출
matplotlib.rc('font', family=font)  # matplotlib 기본 폰트를 한글 폰트로 설정

### 생선 이진분류 - 도미(bream) or 빙어(smelt)

**이진분류란?**
- 두 개의 클래스 중 하나를 선택하는 분류 문제
- 0과 1로 구분하여 표현
- 도미인가? (1), 도미가 아닌가? (0)

**데이터 설명:**
- **특성(Feature)**: 길이(length), 무게(weight) - 2차원 입력 데이터
- **라벨(Label)**: 도미(1) 또는 빙어(0) - 정답 데이터
- **목표**: 새로운 생선의 길이와 무게를 보고 도미인지 빙어인지 예측

In [ ]:
# 도미 데이터 준비
# 도미의 길이 데이터 (cm 단위) - 35개의 샘플
bream_length = [25.4, 26.3, 26.5, 29.0, 29.0, 29.7, 29.7, 30.0, 30.0, 30.7, 31.0, 31.0, 31.5, 32.0, 32.0, 32.0, 33.0, 33.0, 33.5, 33.5, 34.0, 34.0, 34.5, 35.0, 35.0, 35.0, 35.0, 36.0, 36.0, 37.0, 38.5, 38.5, 39.5, 41.0, 41.0]

# 도미의 무게 데이터 (g 단위) - 35개의 샘플
bream_weight = [242.0, 290.0, 340.0, 363.0, 430.0, 450.0, 500.0, 390.0, 450.0, 500.0, 475.0, 500.0, 500.0, 340.0, 600.0, 600.0, 700.0, 700.0, 610.0, 650.0, 575.0, 685.0, 620.0, 680.0, 700.0, 725.0, 720.0, 714.0, 850.0, 1000.0, 920.0, 955.0, 925.0, 975.0, 950.0]

# 데이터 개수 확인 (길이와 무게 데이터의 개수가 일치하는지 확인)
len(bream_length), len(bream_weight)  # (35, 35) - 35개의 도미 데이터

In [ ]:

smelt_length = [9.8, 10.5, 10.6, 11.0, 11.2, 11.3, 11.8, 11.8, 12.0, 12.2, 12.4, 13.0, 14.3, 15.0]


smelt_weight = [6.7, 7.5, 7.0, 9.7, 9.8, 8.7, 10.0, 9.9, 9.8, 12.2, 13.4, 12.2, 19.7, 19.9]


len(smelt_length), len(smelt_weight)  # (14, 14) - 14개의 빙어 데이터

In [ ]:
plt.scatter(bream_length, bream_weight, label='도미', color='red', alpha=0.7)  # 도미 데이터 (빨간색)
plt.scatter(smelt_length, smelt_weight, label='빙어', color='blue', alpha=0.7)  # 빙어 데이터 (파란색)
plt.xlabel('길이 (cm)')  # x축 라벨
plt.ylabel('무게 (g)')   # y축 라벨
plt.title('생선 길이와 무게의 관계')  # 그래프 제목
plt.legend()  # 범례 표시
plt.grid(True, alpha=0.3)  # 격자 표시
plt.show()


##### 데이터 전처리 (Data Preprocessing)

**머신러닝 모델이 원하는 형태로 데이터를 가공하는 과정**

**입력 데이터 (특성, Feature):**
- 형태: [길이, 무게] 2차원 배열 (샘플 수 × 특성 수)
- 예시: [[25.4, 242.0], [26.3, 290.0], ...]

**출력 데이터 (라벨, Label):**
- 형태: 도미 or 빙어 → 1, 0으로 구성된 1차원 배열
- 예시: [1, 1, 1, ..., 0, 0, 0, ...] (도미=1, 빙어=0)

**전처리의 중요성:**
- 머신러닝 알고리즘은 숫자 데이터만 처리 가능
- 일관된 데이터 형태로 변환 필요
- 데이터 품질 향상으로 모델 성능 개선

In [ ]:
fish_length = bream_length + smelt_length  # 도미 길이 + 빙어 길이 = 전체 길이 데이터
fish_weight = bream_weight + smelt_weight  # 도미 무게 + 빙어 무게 = 전체 무게 데이터
len(fish_length), len(fish_weight)  # (49, 49) - 총 49개의 생선 데이터


In [ ]:

train_input = []  # 빈 리스트 초기화
for l, w in zip(fish_length, fish_weight):  
    train_input.append([l, w]) 
    
train_input

In [ ]:

train_label = [1] * len(bream_length) + [0] * len(smelt_length)  # 도미=1, 빙어=0

train_label

##### 🤖 K-최근접 이웃 분류 모델 (K-Nearest Neighbors Classifier)

**KNN 알고리즘의 작동 원리:**
1. 새로운 데이터 포인트가 주어지면
2. 기존 훈련 데이터 중에서 가장 가까운 k개의 이웃을 찾음
3. 이웃들의 클래스를 확인하여 다수결로 예측
4. 거리 측정: 유클리드 거리 사용

**장점:**
- 간단하고 직관적인 알고리즘
- 훈련 과정이 빠름 (데이터를 단순히 저장)
- 비선형 패턴도 잘 처리

**단점:**
- 예측 시 모든 훈련 데이터와의 거리 계산 필요
- 메모리 사용량이 많음
- 차원의 저주 문제

In [ ]:
# K-최근접 이웃 분류기 import
from sklearn.neighbors import KNeighborsClassifier  # scikit-learn의 KNN 분류기 클래스

In [ ]:
kn = KNeighborsClassifier()

kn.fit(train_input, train_label)    
kn.score(train_input, train_label)   
kn.predict([[30, 500], [10, 5]])

- distances : 모델 데이터포인트와의 거리 (유클리드 거리 측정 방식)
- index : 모델 데이터포인트의 인덱스

In [ ]:

distances, index = kn.kneighbors([[30, 500]])  
print("거리:", distances)
print("인덱스:", index) 

print("이웃 데이터:", np.array(train_input)[index])
print("이웃 라벨:", np.array(train_label)[index]) 

In [ ]:

plt.scatter(bream_length, bream_weight, label='도미', color='red', alpha=0.7)  
plt.scatter(smelt_length, smelt_weight, label='빙어', color='blue', alpha=0.7)  
plt.xlabel('길이 (cm)') 
plt.ylabel('무게 (g)')   
plt.title('생선 길이와 무게의 관계') 
plt.legend() 
plt.grid(True, alpha=0.3)  

In [ ]:
train_input = np.array(train_input)
print("훈련 데이터 형태:", train_input.shape)
print("이웃 인덱스:", index)

plt.figure(figsize=(10, 8))
plt.scatter(bream_length, bream_weight, label='도미', color='red', alpha=0.7, s=50)  # 도미 데이터
plt.scatter(smelt_length, smelt_weight, label='빙어', color='blue', alpha=0.7, s=50)  # 빙어 데이터
plt.scatter(30, 500, label='예측 데이터', marker='^', color='green', s=100)  # 예측할 데이터
plt.scatter(train_input[index, 0], train_input[index, 1], label='최근접이웃', marker='x', color='orange', s=100)  # 최근접 이웃들
plt.xlabel('길이 (cm)')
plt.ylabel('무게 (g)')
plt.title('KNN 알고리즘 작동 원리 시각화')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

##### 훈련데이터-테스트(평가)데이터 분리

In [ ]:
fish_input = []
for l, w in zip(fish_length, fish_weight):
    fish_input.append([l, w])

fish_label = [1] * len(bream_length) + [0] * len(smelt_length)

In [ ]:
# sampling 편향 : 데이터가 한쪽으로 치우친 현상
# train 데이터는 전부 다 도미, test 데이터는 전부 다 빙어
train_input = fish_input[:35]
train_label = fish_label[:35]    # 1

test_input = fish_input[35:]
test_label = fish_label[35:]     # 0

kn = KNeighborsClassifier()
kn.fit(train_input, train_label)
kn.score(test_input, test_label)

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_label, test_label = \
train_test_split(fish_input, fish_label, test_size=0.3, stratify=fish_label, random_state=42)

train_input, test_input, train_label, test_label
sum(train_label), sum(test_label)

In [ ]:
train_input = np.array(train_input)
test_input = np.array(test_input)

# 데이터 분포 확인 (산점도)
plt.scatter(train_input[:, 0], train_input[:, 1], label='훈련')
plt.scatter(test_input[:, 0], test_input[:, 1], label='평가')
plt.xlabel('length')
plt.ylabel('weight')
plt.legend()
plt.show()

In [ ]:
kn.fit(train_input, train_label)
kn.score(test_input, test_label)

##### 정규화
- 데이터포인트의 값의 범위를 재정의하는 것
    - StandardScaler
    - MinMaxScaler

In [ ]:
kn = KNeighborsClassifier()
kn.fit(train_input, train_label)
kn.score(test_input, test_label)

predict_input = [25, 150]
kn.predict([predict_input])

In [ ]:
# 데이터 분포 확인 (산점도)
plt.scatter(train_input[:, 0], train_input[:, 1], label='훈련')
plt.scatter(test_input[:, 0], test_input[:, 1], label='평가')
plt.scatter(predict_input[0], predict_input[1], label='예측', marker='D')
plt.xlabel('length')
plt.ylabel('weight')
plt.xlim((0, 1000))
plt.legend()
plt.show()

In [ ]:
distances, index = kn.kneighbors([predict_input])
np.array(train_label)[index]

##### 표준 점수 (Z-Score)
- 평균에서 얼마나 떨어져 있는지를 표준편차 단위로 계산한 값
$$
    표준 점수 = \frac{데이터포인트 - 평균}{표준편차}
$$

In [ ]:
mean = np.mean(train_input, axis=0)
std = np.std(train_input, axis=0)
mean, std

In [ ]:
train_scaled = (train_input - mean) / std
train_scaled

In [ ]:
test_scaled = (test_input - mean) / std
test_scaled

In [ ]:
kn.fit(train_scaled, train_label)
kn.score(test_scaled, test_label)

In [ ]:
predict_scaled = (predict_input - mean) / std
predict_scaled

In [ ]:
kn.predict([predict_input])

In [ ]:
# 데이터 분포 확인 (산점도)
plt.scatter(train_scaled[:, 0], train_scaled[:, 1], label='훈련')
plt.scatter(test_scaled[:, 0], test_scaled[:, 1], label='평가')
plt.scatter(predict_scaled[0], predict_scaled[1], label='예측', marker='D')
plt.xlabel('length')
plt.ylabel('weight')
plt.legend()
plt.show()

**StandardScaler란?** 
- **StandardScaler**는 데이터의 각 특성(feature)을 평균이 0, 표준편차가 1이 되도록 변환(표준화, Standardization)하는 전처리 도구입니다.
- 데이터의 분포를 표준정규분포(평균 0, 표준편차 1)로 맞춰줌으로써, <br>
     모든 특성이 동일한 스케일을 가지게 합니다.



### 2. 표준화(standardization) 공식

$$
z = \frac{x - \mu}{\sigma}
$$

- $x$ : 원래 데이터 값
- $\mu$ : 특성(feature)의 평균
- $\sigma$ : 특성(feature)의 표준편차

---

### 3. 효과

- 표준화를 거치면, 특성의 단위와 값의 크기가 달라도 <br>
동일한 기준에서 모델을 학습시킬 수 있어 알고리즘의 성능이 좋아질 수 있습니다.

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
train_scaled = std_scaler.fit_transform(train_input)
train_scaled

---

### 다중분류
**다중분류(Multi-class Classification) 는** 
- 입력 데이터가 여러 개의 클래스(범주) 중 하나의 클래스에 속하도록 예측하는 문제
<br>

> 💡 즉, “A냐 B냐”처럼 2가지가 아닌
> - “A, B, C, … 중 하나”를 예측하는 문제입니다.

In [ ]:
df = pd.read_csv('./data/fish.csv')
df

In [ ]:
df['Species'].value_counts()

In [ ]:
df['Species'].unique()

In [ ]:
# df[df['Weight'] == 0].index    # Weight가 0인 생선 = 40번 index
df = df.drop(40).reset_index(drop=True)
df.describe()

##### 데이터 전처리
- 훈련 데이터의 규격 : input + label (numpy ndarray)
- 훈련 데이터/테스트(평가) 데이터 분리
- 정규화 (컬럼값 사이의 차이를 완화)

In [ ]:
fish_input = df[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_label = df['Species'].to_numpy()

fish_input.shape, fish_label.shape

In [ ]:
train_input, test_input, train_label, test_label = \
    train_test_split(fish_input, fish_label, test_size=.10, random_state=0)

train_input.shape, test_input.shape, train_label.shape, test_label.shape

In [ ]:
# 정규화
standard_scaler= StandardScaler()
standard_scaler.fit(train_input)   
train_scaled = standard_scaler.transform(train_input)    
test_scaled = standard_scaler.transform(test_input)      

##### 훈련

In [ ]:
kn = KNeighborsClassifier(n_neighbors=7)
kn.fit(train_scaled, train_label)

In [ ]:
kn._fit_X

In [ ]:
kn.classes_

##### 평가

In [ ]:
kn.score(test_scaled, test_label)

##### 다중분류 작동 방식
- 다중분류 클래스 값의 크기와 같은 배열 형태로 결과를 도출

In [ ]:
# 예측 Prediction
kn.predict(test_scaled[5:10])

In [ ]:
print(kn.classes_)
kn.predict_proba(test_scaled[5:10])

In [ ]:
distance, index = kn.kneighbors(test_scaled[7:8])
distance, index

In [ ]:
train_label[index]